In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, concatenate, Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     #brightness_range=[0.8,1.2], 
                                     shear_range=10)

In [4]:
x1 = train.drop(['id', 'digit', 'letter'], axis=1).values
x1 = x1.reshape(-1, 28, 28, 1)
x1 = x1/255
x1_remake = []
for i in range(x1.shape[0]):
    num_aug = 0
    tmp = x1[i]
    tmp = tmp.reshape((1,) + tmp.shape)
    for x_aug in image_generator.flow(tmp, batch_size = 1) :
        if num_aug >= 1:
            break
        x1_remake.append(x_aug[0])
        num_aug += 1
x1_remake = np.array(x1_remake)

x1_total = np.concatenate((x1, x1_remake), axis=0)
print(x1_total.shape)

(4096, 28, 28, 1)


In [5]:
y1_data = train['digit']
y1 = np.zeros((len(y1_data), len(y1_data.unique())))
for i, digit in enumerate(y1_data):
    y1[i, digit] = 1

y1_remake = y1.copy()
y1_total = np.concatenate((y1, y1_remake), axis=0)
print(y1_total.shape)

(4096, 10)


In [6]:
x1_let = train['letter'].values
x1_let = x1_let[:, np.newaxis]
en = OneHotEncoder()
x1_let = en.fit_transform(x1_let).toarray()

x1_remake_let = x1_let.copy()

x1_letter_total = np.concatenate((x1_let, x1_remake_let), axis=0)
x1_letter_total.shape

(4096, 26)

In [7]:
x1_train, x1_val, y1_train, y1_val = train_test_split(x1_total, y1_total, test_size=0.2, shuffle=True, stratify=y1_total)

print(x1_train.shape)
print(x1_val.shape)
print(y1_train.shape)
print(y1_val.shape)

x1_letter_train = x1_letter_total[:x1_train.shape[0],:]
x1_letter_val = x1_letter_total[x1_train.shape[0]:,:]
print(x1_letter_train.shape)
print(x1_letter_val.shape)

(3276, 28, 28, 1)
(820, 28, 28, 1)
(3276, 10)
(820, 10)
(3276, 26)
(820, 26)


In [8]:
x2_train = np.reshape(x1_train, (x1_train.shape[0], x1_train.shape[1], x1_train.shape[2]))
x2_val = np.reshape(x1_val, (x1_val.shape[0], x1_val.shape[1], x1_val.shape[2]))
y2_train = y1_train.copy()
y2_val = y1_val.copy()
x2_letter_train = x1_letter_train.copy()
x2_letter_val = x1_letter_val.copy()

print(x2_train.shape)
print(x2_val.shape)
print(y2_train.shape)
print(y2_val.shape)
print(x2_letter_train.shape)
print(x2_letter_val.shape)

(3276, 28, 28)
(820, 28, 28)
(3276, 10)
(820, 10)
(3276, 26)
(820, 26)


In [9]:
input1 = Input(shape=(28,28,1))
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(input1)
x1 = Dropout(0.2)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.2)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(128, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.2)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Flatten()(x1)

input2 = Input(shape=(26,))
merge1 = concatenate([x1, input2])

x2 = Dense(500, activation='relu')(merge1)
x2 = Dropout(0.2)(x2)
x2 = Dense(100, activation='relu')(x2)

input3 = Input(shape=(28,28))
x3 = LSTM(512, activation='relu', return_sequences=True)(input3)
x3 = Dropout(0.2)(x3)
x3 = LSTM(256, activation='relu', return_sequences=True)(x3)
x3 = Dropout(0.2)(x3)
x3 = LSTM(128, activation='relu')(x3)
x3 = Dropout(0.2)(x3)
x3 = Dense(100, activation='relu')(x3)

input4 = Input(shape=(26,))
merge2 = concatenate([x3, input4])

x4 = Dense(100, activation='relu')(merge2)

merge = concatenate([x2, x4])

x5 = Dense(100, activation='relu')(merge)
x5 = Dropout(0.2)(x5)
x5 = Dense(50, activation='relu')(x5)
x5 = Dropout(0.2)(x5)
outputs = Dense(10, activation='softmax')(x5)

model = Model(inputs = [input1, input2, input3, input4], outputs = outputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 64)   320         input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 28, 28, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 14, 14, 64)   0           dropout[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (

In [12]:
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy') 
              #loss_weights = [0.8, 0.2])
history = model.fit([x1_train, x1_letter_train, x2_train, x2_letter_train], y1_train,
                    validation_data=([x1_val, x1_letter_val, x2_val, x2_letter_val], y1_val), 
                    batch_size=64, epochs=50, verbose=1, 
                    callbacks = [ModelCheckpoint('./models/{epoch:02d}-{val_acc:.4f}.h5',
                    monitor='val_acc', verbose=1, save_best_only=True, mode='auto')])

Train on 3276 samples, validate on 820 samples
Epoch 1/50
3276/3276 [==============================] - 19s 6ms/step - loss: 2.2194 - acc: 0.1697 - val_loss: 2.2004 - val_acc: 0.1902

Epoch 00001: val_acc improved from -inf to 0.19024, saving model to ./models/01-0.1902.h5
Epoch 2/50
3276/3276 [==============================] - 14s 4ms/step - loss: 2.0132 - acc: 0.2552 - val_loss: 1.9845 - val_acc: 0.3427

Epoch 00002: val_acc improved from 0.19024 to 0.34268, saving model to ./models/02-0.3427.h5
Epoch 3/50
3276/3276 [==============================] - 15s 5ms/step - loss: 1.7747 - acc: 0.3599 - val_loss: 1.7686 - val_acc: 0.4037

Epoch 00003: val_acc improved from 0.34268 to 0.40366, saving model to ./models/03-0.4037.h5
Epoch 4/50
3276/3276 [==============================] - 14s 4ms/step - loss: 1.5710 - acc: 0.4545 - val_loss: 1.5832 - val_acc: 0.4793

Epoch 00004: val_acc improved from 0.40366 to 0.47927, saving model to ./models/04-0.4793.h5
Epoch 5/50
3276/3276 [==================


Epoch 00040: val_acc did not improve from 0.73171
Epoch 41/50
3276/3276 [==============================] - 15s 5ms/step - loss: 0.0884 - acc: 0.9762 - val_loss: 1.0103 - val_acc: 0.7280

Epoch 00041: val_acc did not improve from 0.73171
Epoch 42/50
3276/3276 [==============================] - 15s 5ms/step - loss: 0.0866 - acc: 0.9753 - val_loss: 1.0538 - val_acc: 0.7207

Epoch 00042: val_acc did not improve from 0.73171
Epoch 43/50
3276/3276 [==============================] - 15s 4ms/step - loss: 0.0777 - acc: 0.9777 - val_loss: 1.1627 - val_acc: 0.7146

Epoch 00043: val_acc did not improve from 0.73171
Epoch 44/50
3276/3276 [==============================] - 15s 4ms/step - loss: 0.0850 - acc: 0.9737 - val_loss: 1.0638 - val_acc: 0.7317

Epoch 00044: val_acc improved from 0.73171 to 0.73171, saving model to ./models/44-0.7317.h5
Epoch 45/50
3276/3276 [==============================] - 14s 4ms/step - loss: 0.0896 - acc: 0.9722 - val_loss: 1.0437 - val_acc: 0.7317

Epoch 00045: val_acc 

In [ ]:
#from tensorflow.keras.models import load_model
#best_model = load_model('./models/46-0.7463.h5')

In [14]:
x1_test = test.drop(['id', 'letter'], axis=1).values
x1_test = x1_test.reshape(-1, 28, 28, 1)
x1_test = x1_test/255

x2_test = test.drop(['id', 'letter'], axis=1).values
x2_test = x2_test.reshape(-1, 28, 28)
x2_test = x2_test/255

x1_letter_test = test['letter'].values
x1_letter_test = x1_letter_test[:, np.newaxis]
en = OneHotEncoder()
x1_letter_test = en.fit_transform(x1_letter_test).toarray()

x2_letter_test = x1_letter_test.copy()

y1_test = model.predict([x1_test, x1_letter_test, x2_test, x2_letter_test])
y_1 = np.argmax(y1_test, axis=1)
#y_2 = np.argmax(y2_test, axis=1)
print(y_1)
#print(y_2)

[6 9 2 ... 6 8 0]


In [ ]:
#submission = pd.read_csv('data/submission.csv')
#submission['digit'] = y_1

In [ ]:
#submission.to_csv('data/submission(val7463).csv', index=False)

In [ ]:
#submission['digit'] = y_2
#submission.to_csv('data/submission2.csv', index=False)